In [1]:
print('Downloading data')
print('...')
print()
!wget http://hck.re/kkBIfM
print('Data successfully downloaded')
print('##########################################################')

print('Loading Libraries..')
import zipfile
import os,shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop





print('EXTRACTING FILES')
with zipfile.ZipFile('/content/kkBIfM') as f:
  f.extractall()
  
with zipfile.ZipFile('/content/DataSet/Train Images.zip') as f:
  f.extractall()
print('# of train images:', len(os.listdir('/content/Train Images/Large'))+\
      len(os.listdir('/content/Train Images/Small'))
     )
  
with zipfile.ZipFile('/content/DataSet/Test Images.zip') as f:
  f.extractall()
  
print('# of test images:',len(os.listdir('/content/Test Images')))
print()

print('Creating Validation dir.')
print()
train_dir = '/content/Train Images/'
test_dir = '/content/Test Images/'
!mkdir '/content/Validation Images'
val_dir = '/content/Validation Images/'
!mkdir '/content/Validation Images/Large/'
!mkdir '/content/Validation Images/Small/'
index = np.random.choice(range(5999),size=[1000],replace=False)
val_small = np.array(os.listdir(train_dir+'Small'))[index]
#val_small
val_large =  np.array(os.listdir(train_dir+'Large'))[index]


for i in val_small:
  shutil.move(train_dir+'Small/'+i,val_dir+'Small/'+i)
  
for i in val_large:
  shutil.move(train_dir+'Large/'+i,val_dir+'Large/'+i)
  


print('Loading train generator')
print()

train_gen = ImageDataGenerator(rescale=1./255.)
train_generator = train_gen.flow_from_directory(
        train_dir, 
        target_size=(150,150 ),
        shuffle = True,
        batch_size=50,
        class_mode='binary')

print('Loading validation generator')
print()

val_gen = ImageDataGenerator(rescale=1./255.)
val_generator = train_gen.flow_from_directory(
        val_dir, 
        target_size=(150,150 ),
        shuffle = True,
        batch_size=50,
        class_mode='binary')

print('Downloading Inception V3 weights..')
print()
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
print('##########################################################')

print('INCEPTION V3...')  
print()
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  

last_layer = pre_trained_model.get_layer('mixed7')
#print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)  


model = Model( pre_trained_model.input, x)
print('Hello world')
print()

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])
print('Model compiled')
print()

history = model.fit_generator(
      train_generator,
      validation_data= val_generator,   
      steps_per_epoch=20,  
      epochs=10,
      verbose=2)
print('Model training completed')
print('##########################################################')

print('Moving test files to "test" dir.')

print()
test_dir = '/content/Test Images/'

!mkdir '/content/Test Images/test'
print('Test Data generator director has been created; ',test_dir+'test')
print()


for i in os.listdir(test_dir):
  if i!='test':
    shutil.move(test_dir+i,test_dir+'test/'+i)

print('Testing..')
print()
test_dir = '/content/Test Images/'
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        shuffle = False,
        batch_size=32, 
        class_mode=None,)

filenames = test_generator.filenames
nb_samples = len(filenames)
print('Prediction')
predict = model.predict_generator(test_generator,np.ceil(nb_samples/32))
predicted_classes = [1 if i>0.5 else 0 for i in predict]


print('Processing output..')
print()
keys = os.listdir(test_dir+'test')
keys = [i for i in keys if i!='.ipynb_checkpoints']

df = pd.read_csv('/content/DataSet/test.csv')
test_files = df.Image_File.tolist()
_ = []
for i in test_files:
  
  if i in keys:
    _.append(i)
  else:
    break;
    print('ERROR: MISSING FILE !')
    
dic = dict(zip(_,predicted_classes))
df['Class'] = df['Image_File'].apply(lambda x : dic[x])
df.to_csv('submission_file.csv',index=False)
print('Test predictions saved to submission_file.csv')
df


    


...

--2019-10-27 02:03:46--  http://hck.re/kkBIfM
Resolving hck.re (hck.re)... 18.136.192.20, 54.169.227.143, 54.169.47.64
Connecting to hck.re (hck.re)|18.136.192.20|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Cookie coming from hck.re attempted to set domain to hackerearth.com
Cookie coming from hck.re attempted to set domain to hackerearth.com
Cookie coming from hck.re attempted to set domain to hackerearth.com
Location: https://he-public-data.s3-ap-southeast-1.amazonaws.com/DataSet.zip [following]
--2019-10-27 02:03:47--  https://he-public-data.s3-ap-southeast-1.amazonaws.com/DataSet.zip
Resolving he-public-data.s3-ap-southeast-1.amazonaws.com (he-public-data.s3-ap-southeast-1.amazonaws.com)... 52.219.128.11
Connecting to he-public-data.s3-ap-southeast-1.amazonaws.com (he-public-data.s3-ap-southeast-1.amazonaws.com)|52.219.128.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733209860 (699M) [application/zip]
Saving t

EXTRACTING FILES
# of train images: 11998
# of test images: 7534

Creating Validation dir.

Loading train generator

Found 9998 images belonging to 2 classes.
Loading validation generator

Found 2000 images belonging to 2 classes.

--2019-10-27 02:05:18--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 2a00:1450:400c:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  30.5MB/s    in 2.7s    

2019-10-27 02:05:21 (30.5 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

##########################################################
INCEPTION V3.

,Image_File,Class
0,lg 988 (1).png,1
1,lg 988 (10).png,0
2,lg 988 (100).png,0
3,lg 988 (101).png,0
4,lg 988 (102).png,0
...,...,...
7529,lg99 (95).png,0
7530,lg99 (96).png,0
7531,lg99 (97).png,0
7532,lg99 (98).png,1
